# Technical Report

<table>
    <tr>
        <td> Topic: A Star Search Algorithm</td>
    </tr>
    <tr>
        <td>Submitted By: </td>
        <td>A.Faiyaz</td>
    </tr>
    <tr>
        <td>Submitted To: </td>
        <td>Dr. Nasima Begum</td>
    </tr>
</table>

# Introduction:
The A* search algorithm is a popular path-finding algorithm that is widely used in artificial intelligence and computer science. It is a type of informed search algorithm that combines the benefits of both Dijkstra's algorithm and Greedy Best-First-Search. In this assignment I will use A* search algorithm to find the shortest path from my home to UAP.
I will use my very own dataset with location name,longitude,lattitude and graph dataset where location are connected by edges each edge represents path to go from location A to location B


At first we will import some necessary packages to work with

In [1]:
import pandas as pd
from haversine import haversine, Unit
import folium
from folium.features import DivIcon

We neeed  to import our location data which contains location name,lattatiude and longitude.

In [2]:
location_data=pd.read_csv('./location_data.csv')

Now lets have a look at our dataset

In [3]:
location_data

,Location,Lat,Lon
0,Rayer Bazar,23.743654,90.362028
1,A,23.745726,90.365354
2,B,23.745712,90.365448
3,C,23.749129,90.363855
4,D,23.750773,90.368134
5,E,23.751802,90.367650
6,F,23.756464,90.374950
7,G,23.758456,90.374806
8,H,23.759085,90.389672
9,I,23.757099,90.390388


In [4]:
source=location_data.iloc[0]
goal=location_data.iloc[-1]
print(source)
print(goal)

Location    Rayer Bazar
Lat           23.743654
Lon           90.362028
Name: 0, dtype: object
Location          UAP
Lat         23.754983
Lon         90.389014
Name: 16, dtype: object


### MAP
Lets plot our data using folium library and marker green indicates starting position and pink indicates destination

In [5]:
location_data_excluded=location_data.iloc[2:location_data.shape[0]-1]

In [6]:
m = folium.Map(location=[23.745268183480682,90.36089271175031], zoom_start=12, tiles="Stamen Terrain")

points=[]

folium.Marker(
            [source[1],source[2]], popup=source[0], tooltip=source[0],
            icon=folium.Icon(color='green')).add_to(m)

folium.Marker(
            [goal[1],goal[2]], popup=goal[0], tooltip=goal[0],
            icon=folium.Icon(color='pink')).add_to(m)



for i in range(0,location_data_excluded.shape[0]):
        folium.Marker(
            [location_data_excluded.iloc[i][1],location_data_excluded.iloc[i][2]], popup=location_data_excluded.iloc[i][0], tooltip=location_data_excluded.iloc[i][0]
        ).add_to(m)
          
m



In [7]:
m.save(outfile='map.html')

Time to import our graph which is situated in location graph dataset lets import it

In [8]:
location_graph=pd.read_csv('./location_graph.csv')


In [9]:
graph=dict()
path_costs=[]
for i in range(0,location_graph.shape[0]):
    loc_1=location_data[location_data['Location']==location_graph.iloc[i][0]].values.tolist()
    loc_2=location_data[location_data['Location']==location_graph.iloc[i][1]].values.tolist()
    if len(loc_1)>0 and len(loc_2)>0:
        loc_1_name=loc_1[0][0]
        loc_2_name=loc_2[0][0]
        loc_1=loc_1[0][1:]
        loc_2=loc_2[0][1:]
        path_cost=haversine(loc_1, loc_2, unit='km')
        if loc_1_name in graph:
            graph[loc_1_name].append((loc_2_name,path_cost))
            path_costs.append(path_cost)
        else:
            graph[loc_1_name]=[(loc_2_name,path_cost)]
            path_costs.append(path_cost)

Lets have a look at this

In [10]:
location_graph["Path Cost(KM)"]=path_costs
location_graph.to_csv('./location_map.csv')
location_graph

,Location_A,Location_B,Path Cost(KM)
0,Rayer Bazar,A,0.409588
1,A,B,0.009669
2,B,C,0.413053
3,C,D,0.472333
4,D,E,0.124601
5,E,F,0.905910
6,F,G,0.222020
7,G,H,1.514496
8,H,I,0.232601
9,I,UAP,0.273680


# Heuristic Function:
In A* search, a heuristic function is used to estimate the distance or cost from the current node to the goal node. This heuristic function is denoted by h(n) and is defined as:

h(n) = estimated cost from node n to the goal node

***Spherical Law of Cosine as Heuristic function:***
The spherical law of cosines is a formula used to calculate the shortest distance between two points on a sphere (such as the Earth). This formula can be used as a heuristic function in the A* search algorithm to estimate the distance between the current node and the goal node.

The formula for the spherical law of cosines to calculate the distance (d) between two points on the surface of the Earth, given their latitudes and longitudes in radians, is:

$$\cos(d) = \sin(\phi_1) \sin(\phi_2) + \cos(\phi_1) \cos(\phi_2) \cos(\Delta \lambda)$$

where $\phi_1$ and $\phi_2$ are the latitudes of the two points, and $\Delta \lambda$ is the difference in their longitudes. The distance (d) between the two points can be calculated by rearranging the equation and solving for d:

$$d = \cos^{-1}\left(\sin(\phi_1) \sin(\phi_2) + \cos(\phi_1) \cos(\phi_2) \cos(\Delta \lambda)\right)$$

Note that the latitudes and longitudes must be in radians.

To use this formula as a heuristic function in A* search, we would need to calculate the distance between the current node and the goal node using the above formula, where the current node's latitude and longitude would be used for lat1 and lon1, and the goal node's latitude and longitude would be used for lat2 and lon2. This distance value would then be used as the estimated cost from the current node to the goal node.

It's worth noting that the spherical law of cosines heuristic function may not be admissible in all cases, as it can sometimes overestimate the actual cost of reaching the goal node. This can happen if the path between the current node and the goal node includes a large change in latitude or longitude. 

In [11]:
import math
def heuristic_function(loc_1,loc_2):
    '''
    Used spherical law of cosine for the heuristic function
    '''
    R = 6371  # radius of the Earth in kilometers
    lat1 = loc_1[0]  
    lat2 = loc_2[0]  
    lon1 = loc_1[1]  
    lon2 = loc_2[1]  

    phi1 = lat1 * math.pi / 180
    phi2 = lat2 * math.pi / 180
    delta_lambda = (lon2 - lon1) * math.pi / 180

    d = math.acos(
        math.sin(phi1) * math.sin(phi2) + math.cos(phi1) * math.cos(phi2) * math.cos(delta_lambda)
    ) * R
    return d

Adding the above heurisitic function to our dataset

In [12]:
heuristics =dict()
goal_loc=(float(goal[1]),float(goal[2]))
hs=[]
for i in range(0,location_data.shape[0]):
    hs.append(heuristic_function(goal_loc,(float(location_data.iloc[i][1]),float(location_data.iloc[i][2]))))
    heuristics[location_data.iloc[i][0]]=hs[-1]

Let us have a look on the dataset with heurisitics

In [13]:
location_data["heurisitics"]=hs
location_data

,Location,Lat,Lon,heurisitics
0,Rayer Bazar,23.743654,90.362028,3.021733
1,A,23.745726,90.365354,2.618773
2,B,23.745712,90.365448,2.610633
3,C,23.749129,90.363855,2.642020
4,D,23.750773,90.368134,2.175994
5,E,23.751802,90.367650,2.202880
6,F,23.756464,90.374950,1.440780
7,G,23.758456,90.374806,1.496587
8,H,23.759085,90.389672,0.461020
9,I,23.757099,90.390388,0.273679


# Graph visualization with heuristics and  path costs 

In [14]:
from pyvis.network import Network
import pandas as pd
import networkx as nx
net=Network(notebook=True)
G = nx.Graph()
for location in location_data.values.tolist():
    loc_label=location[0]+' h(n) = '+str(round(heuristics[location[0]],3))
    if location[0]==source[0]:
        #net.add_node(location[0],label=str(location[0]),color='blue')
        G.add_node(location[0],color="green",label=loc_label)
    elif location[0]==goal[0]:
        #net.add_node(location[0],label=str(location[0]),color='green')
        G.add_node(location[0],color="magenta",label=loc_label)
    else:
        #net.add_node(location[0],label=str(location[0]))
        G.add_node(location[0],label=loc_label)

for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    G.add_edge(locationA,locationB,weight=float(location[2])*10, title=str(location[2]), label=str(round(location[2],5))+'KM')
    #G[locationA][locationB]['weight']=str(location[2])
    #net.add_edge(locationA,locationB,value=str(location[2]),title=str(location[2])+' KM')
net.from_nx(G,show_edge_weights=True)
net.toggle_physics(False)
net.show(name='graph.html',notebook=True)
    


graph.html


Green Indicates starting position which is in my case 'Rayer Bazar' and magenta indicates UAP.

# The A * Search Algorithm

The A* search algorithm is a widely used algorithm in artificial intelligence and computer science for finding the shortest path between two points on a graph. It is an extension of the Dijkstra's algorithm, which is a popular algorithm for finding the shortest path between two points in a graph with non-negative edge weights.

Here are the steps involved in the A* search algorithm:

    1) Initialize the start node and set its cost to zero.
    2) Initialize an open list to contain the start node.
    3) Initialize a closed list to be empty.
    4) While the open list is not empty, do the following:
        a. Pop the node with the lowest cost from the open list. This is the current node.
        b. If the current node is the goal node, return the path from the start node to the current node.
        c. Add the current node to the closed list.
        d. For each neighbor of the current node, calculate the cost to move from the current node to the neighbor.
        e. If the neighbor is not in the open list or the cost to move to the neighbor is less than the previous cost, update the neighbor's cost and set its parent to the current node.
        f. Add the neighbor to the open list if it's not already there.
    5)If the goal node is not found, then there is no path from the start node to the goal node.

In [15]:
from collections import defaultdict
import heapq
score_graph=defaultdict(int)
def a_star_algorithm(edges:dict[str],
                     heuristic:dict[str],
                     source:str,
                     goal:str)->list[int]:
  
    '''
    A star search algorithm
    takes in edges,heuristic,source and goal 
    returns list and optimal cost
    if nothing is found then returns a empty list
    '''
    g_score=defaultdict(lambda:float('Inf'))
    f_score=defaultdict(lambda:float('Inf'))
    in_the_fringe=defaultdict(lambda:False)

    f_score[source]=heuristic[source]
    g_score[source]=0
    open_fringe=[(f_score[source],source)]
    path=defaultdict(lambda:False)
    close_fringe=[]

    path_found=False
    
    
    optimal_cost=0
    while len(open_fringe)>0:
        
        _,node=heapq.heappop(open_fringe)
        
        if node==goal:
            path_found=True
            break
        if node in edges:
            for adjacent in edges[node]:
            
                adjacent_node,cost_to_adjacent=adjacent
                if g_score[node]+cost_to_adjacent<g_score[adjacent_node]:
                    g_score[adjacent_node]=g_score[node]+cost_to_adjacent
                    score_graph[(adjacent_node,node)]=cost_to_adjacent
                    
                    f_score[adjacent_node]=g_score[adjacent_node]+heuristic[adjacent_node]
                    path[adjacent_node]=node
                    
                    adjacent_data=str(adjacent_node)+str(f_score[adjacent_node])
                    if adjacent_data not in in_the_fringe:
                        in_the_fringe[adjacent_data]=True
                        heapq.heappush(open_fringe,(f_score[adjacent_node],adjacent_node))
        
        #print(f"{node}: {open_fringe}")
    
    if path_found:
        total_path=[]
        source=goal
        total_path.append(goal)
        while path[source]:
            optimal_cost+=score_graph[(source,path[source])]
            source=path[source]
            total_path.append(source)
        total_path.reverse()
        return total_path,optimal_cost
    else:
        return []

***Let us see our graph as python dictionary***

In [16]:
graph

{'Rayer Bazar': [('A', 0.40958790293027125), ('P', 0.30576841014184286)],
 'A': [('B', 0.009668975124229139)],
 'B': [('C', 0.41305312631438906)],
 'C': [('D', 0.47233266697542103)],
 'D': [('E', 0.12460103709072)],
 'E': [('F', 0.9059099628608953)],
 'F': [('G', 0.2220203586536073)],
 'G': [('H', 1.5144961675667254)],
 'H': [('I', 0.2326009290791598)],
 'I': [('UAP', 0.2736796879282156)],
 'P': [('Q', 0.7501990660106809)],
 'Q': [('R', 0.5730082784773147)],
 'R': [('S', 0.43305629084865566)],
 'S': [('T', 0.3842204074633529)],
 'T': [('U', 0.6121674365421983)],
 'U': [('UAP', 0.44834399020045895)]}

***Heurisitics***

In [17]:
heuristics

{'Rayer Bazar': 3.0217332638282914,
 'A': 2.6187727004164363,
 'B': 2.6106333669422837,
 'C': 2.642019722447652,
 'D': 2.1759941838745718,
 'E': 2.2028803751533284,
 'F': 1.4407796029578588,
 'G': 1.4965874176334408,
 'H': 0.46102026409764496,
 'I': 0.2736792985079538,
 'P': 2.753600122757336,
 'Q': 2.078545900269154,
 'R': 1.5215403063624244,
 'S': 1.3055422464831339,
 'T': 1.0587435907940188,
 'U': 0.44834338055038075,
 'UAP': 0.0}

***Goal Node***

In [18]:
goal

Location          UAP
Lat         23.754983
Lon         90.389014
Name: 16, dtype: object

***Starting position***

In [19]:
source

Location    Rayer Bazar
Lat           23.743654
Lon           90.362028
Name: 0, dtype: object

### Passing  all the data to our a_star_algorithm_function to calculate the shortest path

In [34]:
path,optimal_cost=a_star_algorithm(graph,heuristics,source[0],goal[0])
path_="->".join(path)


In [35]:
print(f"Path: {path_}")

Path: Rayer Bazar->P->Q->R->S->T->U->UAP


In [36]:
print(f"Optimal Cost is : {optimal_cost} km")


Optimal Cost is : 3.506763879684504 km


***Visualizing the shortest path***

In [37]:
from pyvis.network import Network
net=Network(notebook=True)
for location in location_data.values.tolist():
    if location[0]==source[0]:
        net.add_node(location[0],label=location[0],color="green")
    
    if location[0]==goal[0]:
        net.add_node(location[0],label=location[0],color="magenta")
    else:
        net.add_node(location[0],label=location[0])
for i in range(1,len(path)):
    net.add_edge(path[i-1],path[i],color="red")
    
for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    net.add_edge(locationA,locationB)
net.show('final_graph.html')

final_graph.html


***More visualization with path cost***

In [38]:
from pyvis.network import Network
net=Network(notebook=True)
G = nx.Graph()
for location in location_data.values.tolist():
    if location[0]==source[0]:
        G.add_node(location[0],label=location[0],color="green")
    
    if location[0]==goal[0]:
        G.add_node(location[0],label=location[0],color="magenta")
    else:
        G.add_node(location[0],label=location[0])
    #net.add_node(location[0],label=location[0])
for i in range(1,len(path)):
    G.add_edge(path[i-1],path[i],color='red',label=str(round(score_graph[(path[i],path[i-1])],5))+' KM')
    #net.add_edge(path[i-1],path[i],color="red")
    
for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    G.add_edge(locationA,locationB)
    #net.add_edge(locationA,locationB)
net.from_nx(G,show_edge_weights=True)
net.show('final_graph.html')

final_graph.html


In [39]:
print(f"Optimal Cost is : {optimal_cost} km")


Optimal Cost is : 3.506763879684504 km
